In [1]:
import pandas as pd
import re

In [153]:
clean_cars=pd.read_csv("clean_car.csv")
car_vins=pd.read_csv("cars2.csv")

In [135]:
clean_cars.drop(columns="Unnamed: 0",inplace=True)

In [19]:

import csv
my_rows=[]
with open('cars3.csv', newline='') as f:
    reader = csv.reader(f)
    for row in reader:
        my_rows.append(row)
df = pd.DataFrame(my_rows, columns=('A','B','C','D','E','F','G','H','I','J','K','l'))

In [20]:
row_remov=df["J"]

In [21]:
style=df['D'][0]

In [22]:
pattern=re.sub(r'\ddr','',style)

In [23]:
engine=clean_cars["Engine"]

In [24]:
#checking for different style of engines
index_cyl=[]
for i in range(len(engine)):
    Char_f=re.compile(r'-')
    Char_F=Char_f.search(engine[i])
    index_cyl.append(Char_F.end())

In [25]:
cy_type=['V','Inline','W','Flat']

In [26]:
new_engine=[]
engine_check=[]
check_append=[]
for j in range(len(engine)):
    engine_tes=engine[j]
    #checking to see what type they classfiy at to change it to cyl and removing anything to see to make it universal 4-cyl 
    for i in range(len(cy_type)):
        Char_V=re.compile(cy_type[i])
        Char_v=Char_V.search(engine_tes)
        try:
            if cy_type[i]==Char_v.group():
                check_append.append(j)
                new_eng=re.sub(cy_type[i]+'-\d',str(engine_tes[index_cyl[j]])+"cyl",engine_tes)
                sec_eng=re.sub(r'charged',"",new_eng)
                third_eng=re.sub(r'CNG',"",sec_eng)
                final_eng=re.sub(r'Gas',"",third_eng)
                engine_check.append(final_eng)
                new_engine.append(final_eng.lower().replace(" ",''))
        except:
            new_eng = None

In [28]:
#checking if a certain index is in the list if not drop it 
drop_index=[]
for i in range(len(check_append)):
    if i not in check_append:
        drop_index.append(i)

In [136]:
clean_cars.drop(index=drop_index,inplace=True)
clean_cars.reset_index(drop=True,inplace=True)

In [31]:
ind_remove= []
row_TvF=pd.notnull(row_remov).tolist()
for i in enumerate(row_TvF):
    if i[1] == True:
        ind_remove.append(i[0])

In [32]:
for i in range(len(ind_remove)):
    ind=ind_remove[i]
    df.drop(ind,inplace=True)

In [33]:
df.reset_index(inplace=True)
df.drop(columns=["index"],inplace=True)

In [34]:
compare_name=df["A"]+df["B"].str.lower()+df["C"].str.lower()
compare_engine=df["E"].str.lower().str.replace(" ","")

In [35]:
clean_style=clean_cars['Style']
clean_key=clean_cars["Year"].astype(str)+clean_cars["Name"].str.lower()
clean_vins=clean_cars["Vin"]
clean_key=clean_key.str.replace(" ","")

In [36]:
#creating a dict base on same name vehicles and seeing what index they happens appends them to a list and add it to dicts 
new_dict={}
for i in range(len(clean_key)):
    for j in range(len(compare_name)):
        if re.search(clean_key[i],compare_name[j]):
            if i in new_dict:
                new_dict[i].append(j)
            else:
                new_dict[i] = [j]

In [37]:
f = open("dict.txt","w")
f.write( str(new_dict) )
f.close()

In [38]:

new_dicts={}
key_list = list(new_dict.keys())
#get keys for dict and get length of vaules as there can be many
for i in range(len(new_dict)):
    dif_engine=new_dict[key_list[i]]
    index_engine=len(dif_engine)
    #compares the engines to see if they are the same if not removes form list in dict
    for j in range(index_engine):
        if re.search(new_engine[key_list[i]],compare_engine[dif_engine[j]]):
            if key_list[i] in new_dicts:
                new_dicts[key_list[i]].append(dif_engine[j])
            else:
                new_dicts[key_list[i]] = [dif_engine[j]]

In [39]:
clean_style=clean_cars['Style']
clean_key=clean_cars["Year"].astype(str)+clean_cars["Name"].str.lower()
clean_vins=clean_cars["Vin"]
clean_key=clean_key.str.replace(" ","")

In [40]:
df_style=df["D"]

In [41]:
key_lists = list(new_dicts.keys())

In [42]:
new_di={}
false_count=[]
Count=[]
#splits the styles to check if they are the same and if not removes for dict

for k in range(len(key_lists)):
    clean_styles=re.split('\s',clean_style[key_lists[k]])
    index_len=len(new_dicts[key_lists[k]])
    for m in range(index_len):
        df_styles=re.split('\s',df_style[new_dicts[key_lists[k]][m]])
        for j in range(len(clean_styles)):
            for i in range(len(df_styles)):
                if clean_styles[j] == df_styles[i]:
                    None
                else:
                    false_count.append("False")
        if j<index_len:
            Count.append(len(false_count))
            false_count.clear()
        elif index_len==1:
            Count.append(len(false_count))
            false_count.clear()
        if index_len==len(Count):
            min_false=min(Count)
            for positions,min_F in enumerate(Count):
                    if min_F==min_false:
                        if key_lists[k] not in new_di:
                            new_di[key_lists[k]] = [new_dicts[key_lists[k]][positions]]
                        else:
                            new_di[key_lists[k]].append(new_dicts[key_lists[k]][positions])
            Count.clear()
    

In [43]:
key_lis = list(new_di.keys())

In [44]:
df_final_engine=df["E"]

In [183]:
final_dict={}
append_false=[]
count=[]
#splits the styles to check if they are the same and if not removes for dict
for k in range(len(key_lis)):
    if len(new_di[key_lis[k]])>1:
            clean_styles=re.split('\s',clean_style[key_lis[k]])
            index_len=len(new_di[key_lis[k]])
            for m in range(index_len):
                df_engines=re.split('\s',df_final_engine[new_di[key_lis[k]][m]])
                for j in range(len(clean_styles)):
                    for i in range(len(df_engines)):
                        if clean_styles[j] == df_engines[i]:
                            None
                        else:
                            append_false.append("False")
                if j<=index_len:
                    count.append(len(append_false))
                    append_false.clear()
                if index_len==len(count):
                    min_false=min(count)
                    for positions,min_F in enumerate(count):
                        if min_F==min_false:
                            if key_lis[k] not in final_dict:
                                final_dict[key_lis[k]] = [new_di[key_lis[k]][positions]]
                            else:
                                final_dict[key_lis[k]].append(new_di[key_lis[k]][positions])
                    count.clear()
    else:
        final_dict[key_list[k]]=new_di[key_lis[k]]

In [46]:
final_keys=list(final_dict.keys())

In [47]:
for k in range(len(final_keys)):
    final_len=len(final_dict[final_keys[k]])    
    if final_len>1:
        del final_dict[final_keys[k]] 

In [49]:
import numpy as np

In [50]:
clean_cars["HP"]=np.nan
clean_cars["Curbwieght"]=np.nan

In [51]:
clean_cars.reset_index(inplace=True)

In [52]:
df.reset_index(inplace=True)

In [54]:
key_append=list(final_dict.keys())

In [87]:
append_df=pd.DataFrame(columns=["HP","Curbwieght",'keys','RPM'])

In [88]:
HP=list(df['F'])
CW=list(df['G'])

In [89]:
#inputing HP based on the index number for final dict
for k in range(len(key_append)):
    index_key=final_dict[key_append[k]]
    for i in range(len(index_key)):
        append_df=append_df.append({'HP':HP[final_dict[key_append[k]][i]] ,'Curbwieght':CW[final_dict[key_append[k]][i]],
                                    'keys':key_append[k]}, ignore_index=True) 

In [91]:
#checking if row had rpm if not add nan
for k in range(len(append_df['HP'])):
    if re.search("rpm",append_df["HP"][k]):
        None
    else:
        append_df.loc[k,"HP"]=np.nan

In [92]:
# checking if lbs is in curbwieght if not add nan if a row did not have lbs
for k in range(len(append_df['Curbwieght'])):
    if re.search("lbs.",append_df["Curbwieght"][k]):
        None
    else:
        append_df.loc[k,"Curbwieght"]=np.nan

In [96]:
Horse_power=append_df["HP"]
Cur_wieght=append_df["Curbwieght"]

In [94]:
#making hp and rpm own columns and removing rpm
for i in range(len(Horse_power)):
    if type(Horse_power[i])==str:
        Char_f=re.compile(r'@')
        Char_F=Char_f.search(Horse_power[i])
        b_rpm=Horse_power[i][Char_F.end():]
        append_rpm=re.sub("rpm","",b_rpm)
        append_df['RPM'].loc[i]=append_rpm
        b_Hp=Horse_power[i][:-Char_F.end()-2]
        append_Hp=re.sub("hp","",b_Hp)
        append_df["HP"].loc[i]=append_Hp
    else:
        None

In [196]:
#removing then lbs to only have number
for i in range(len(Cur_wieght)):
    if type(Cur_wieght[i])==str:
        append_curb=re.sub("lbs.","",Cur_wieght[i])
        append_df['Curbwieght'].loc[i]=append_curb
    else:
        None

In [101]:
append_df.set_index("keys",drop=True,inplace=True)

In [127]:
#merging df
df_merged = clean_cars.merge(append_df, how='outer', left_index=True, right_index=True)

In [154]:
Vin=car_vins['Vin']

In [155]:
#checking if they have vin are same and get index
vin_index=[]
for i in range(len(Vin)):
    try:
        if Vin[i]==Vin[i+1]:
            vin_index.append(i)
            vin_index.append(i+1)
    except:
        if Vin[i]==Vin[i]:
            None

In [156]:
car_vins.drop(index=vin_index,inplace=True)
style=car_vins['Style']

In [157]:
# if same style drop the row
style_list=style.tolist()
for position,char in enumerate(style_list):
    if char=='Style':
        car_vins.drop(index=position,inplace=True)

In [158]:
car_vins.rename(columns={'Style':"Styles"},inplace=True)
new_vin=car_vins.set_index("Vin")

In [167]:
new_cars=pd.merge(df_merged,car_vins, on='Vin', how="left")

In [160]:
new_vins=new_cars["Vin"]

In [161]:

n_vins=[]
for i in range(len(new_cars)):
    try:
        if new_vins[i]==new_vins[i+1]:
            n_vins.append(i+1)
    except:
        if new_vins[i]==new_vins[i]:
            None

In [169]:
new_cars.drop(index=n_vins,inplace=True)

In [170]:
new_cars.reset_index(inplace=True)

In [171]:
new_cars.drop(columns='index',inplace=True)

In [175]:
#removing makes form the name just to get brand
brand=new_cars["Brand"]
for i in range(len(brand)):
    if type(brand[i])==str:
        char_b=re.compile(r"Make: ")
        char_B=char_b.search(brand[i])
        if "Make: "==char_B.group():
            new_brand=re.sub(r'Make: ',"",brand[i])
    else:
        None

In [181]:
new_cars.drop(columns=["level_0","Curbwieght_x","HP_x"],inplace=True)

In [228]:
new_cars["Styles"].unique()

array(['Sport Utility Vehicle (SUV)/Multi-Purpose Vehicle (MPV)',
       'Sedan/Saloon', 'Wagon', 'Hatchback/Liftback/Notchback', 'Pickup',
       'Crossover Utility Vehicle (CUV)', nan, 'Coupe', 'Cargo Van',
       'Convertible/Cabriolet', 'Van', 'Truck', 'Minivan', 'Body Class:',
       'Roadster', 'Sport Utility Truck (SUT)'], dtype=object)

In [252]:
type_vech=["Roadster","Minivan","Convertible","Coupe","Sedan","SUV","Wagon","Hatchback",
            "Truck","CUV","Cargo Van","Van","Pickup","SUT"]
Styles_car=list(new_cars["Styles"])
Make_car= list(new_cars["Brand"])

In [258]:
#remove unwanted things to only have type vehicles and brand
for i in range(len(Styles_car)):
    if type(Styles_car[i])==str:
        append_style=re.sub("Body Class: ","",Styles_car[i])
        for j in range(len(type_vech)):
            if re.match(type_vech[j],append_style):
                new_cars['Styles'].loc[i]=type_vech[j]
            elif re.search(type_vech[j],append_style):
                new_cars['Styles'].loc[i]=type_vech[j]
    if type(Make_car[i])==str:
        append_make=re.sub("Make: ","",Make_car[i])
        new_cars['Brand'].loc[i]=append_make
    else:
        None

In [259]:
new_cars

,Vin,Name,Year,Miles,Location,Extrior,Style,Interior,Engine,Transmission,...,Color,Turbo,Temp,Cylinder,DrivingType,HP_y,Curbwieght_y,RPM,Styles,Brand
0,WP1AA2AY4KDA04144,Porsche Cayenne,2019,5885,Norwood MA,Quartzite Metallic,AWD,Black,3.0L V-6 Gas Turbocharged,Automatic,...,Others,0,V-6 Gas,6,0,340,6239,5300,SUV,PORSCHE
1,19XFC2F50GE218876,Honda Civic,2016,43488,Norwood MA,Taffeta White,LX Sedan CVT,Ivory,2.0L Inline-4 Gas,Automatic,...,White,1,Inline-4,4,1,158,2751,6500,Sedan,HONDA
2,5NPD84LF6HH091405,Hyundai Elantra,2017,26516,Norwood MA,Phantom Black,SE 2.0L Sedan Automatic (alt),Gray,2.0L Inline-4 Gas,Automatic,...,Black,1,Inline-4,4,1,NaN,NaN,NaN,Sedan,HYUNDAI
3,1C4BJWEG2FL581541,Jeep Wrangler,2015,56007,Norwood MA,Billet Silver Metallic Clearcoat,Unlimited Sahara,Black,3.6L V-6 Gas,Manual,...,Silver,1,V-6 Gas,6,2,285,4900,6400,SUV,JEEP
4,JN8AT2MV0HW005533,Nissan Rogue,2017,28985,Norwood MA,Glacier White,S AWD,Charcoal,2.5L Inline-4 Gas,Automatic,...,White,1,Inline-4,4,0,170,4678,6000,Wagon,NISSAN
5,JHMGK5H7XGS013116,Honda Fit,2016,25000,Boston MA,Modern Steel Metallic,EX CVT,Black,1.5L Inline-4 Gas,Automatic,...,Others,1,Inline-4,4,1,130,2630,6600,Hatchback,HONDA
6,JF2SH63629H779227,Subaru Forester,2009,91610,Walpole MA,Sage Green Metallic,2.5X with Premium/All-Weather Package Auto,Platinum Gray,2.5L Flat-4 Gas,Automatic,...,Green,1,Flat-4 G,4,0,NaN,NaN,NaN,Wagon,SUBARU
7,SALVC2RX8JH301698,Land Rover Range Rover Evoque,2018,20473,Norwood MA,Moraine Blue Metallic,Landmark Edition 5-Door,Ebony/Ebony,2.0L Inline-4 Gas Turbocharged,Automatic,...,Blue,0,Inline-4,4,2,NaN,NaN,NaN,SUV,LAND ROVER
8,3N1AB7AP7GY254570,Nissan Sentra,2016,77045,Boston MA,Gun Metallic,S CVT,Charcoal,1.8L Inline-4 Gas,Automatic,...,Others,1,Inline-4,4,1,248,6450,4750,Sedan,NISSAN
9,1FTMF1CWXAKE62092,Ford F-150,2010,95645,West Bridgewater MA,Gray,XL Regular Cab 6.5' Box 2WD,Gray,4.6L V-8 Gas,Automatic,...,Gray,1,V-8 Gas,8,3,270,5840,6200,Pickup,FORD


In [260]:
new_cars.to_csv("new_cars.csv",header=True)